In [1]:
import numpy as np

def compute_weights(sigma, delta):
    # sigma, delta: arrays length N
    tau = np.concatenate(([0], np.cumsum(sigma[:-1] * delta[:-1])))
    T = np.exp(-tau)
    w = T * (1 - np.exp(-sigma * delta))
    return w

def render_depth_and_opacity(sigma, delta, z):
    w = compute_weights(sigma, delta)
    C = np.sum(w)
    depth = np.sum(w * z)
    return C, depth

# Test 1: Single sample
sigma = np.array([2.0])
delta = np.array([1.0])
z = np.array([5.0])  # sample at depth 5
C1, depth1 = render_depth_and_opacity(sigma, delta, z)
expected_C1 = 1 - np.exp(-2.0)
print("Test 1 — Single sample")
print(f"Computed C = {C1:.6f}, depth = {depth1:.6f}")
print(f"Expected C = {expected_C1:.6f}, expected depth = {expected_C1 * z[0]:.6f}\n")

# Test 2: Zero density everywhere
sigma = np.zeros(3)
delta = np.ones(3)
z = np.array([1.0, 2.0, 3.0])
C2, depth2 = render_depth_and_opacity(sigma, delta, z)
print("Test 2 — Zero density")
print(f"Computed C = {C2:.6f}, depth = {depth2:.6f}")
print("Expected C = 0.000000, depth = 0.000000\n")

# Test 3: First sample fully opaque
sigma = np.array([1e6, 1.0])
delta = np.array([0.1, 0.1])
z = np.array([1.0, 2.0])
C3, depth3 = render_depth_and_opacity(sigma, delta, z)
print("Test 3 — First sample opaque")
print(f"Computed C = {C3:.6f}, depth = {depth3:.6f}")
print("Expected C ≈ 1.000000, depth ≈ 1.000000")


Test 1 — Single sample
Computed C = 0.864665, depth = 4.323324
Expected C = 0.864665, expected depth = 4.323324

Test 2 — Zero density
Computed C = 0.000000, depth = 0.000000
Expected C = 0.000000, depth = 0.000000

Test 3 — First sample opaque
Computed C = 1.000000, depth = 1.000000
Expected C ≈ 1.000000, depth ≈ 1.000000


## Experiment of Sigma and its effective on the accaracy of the depth

In [2]:
sigma = np.array([0, 0, 1])
delta = np.ones(3)
z = np.array([1.0, 2.0, 3.0])
C3, depth3 = render_depth_and_opacity(sigma, delta, z)
print("Test — First sample opaque")
print(f"Computed C = {C3:.6f}, depth = {depth3:.6f}")


Test — First sample opaque
Computed C = 0.632121, depth = 1.896362


In [3]:
sigma = np.array([0, 0, 10])
delta = np.ones(3)
z = np.array([1.0, 2.0, 3.0])
C3, depth3 = render_depth_and_opacity(sigma, delta, z)
print("Test — First sample opaque")
print(f"Computed C = {C3:.6f}, depth = {depth3:.6f}")


Test — First sample opaque
Computed C = 0.999955, depth = 2.999864


In [4]:
sigma = np.array([0, 0, 1000])
delta = np.ones(3)
z = np.array([1.0, 2.0, 3.0])
C3, depth3 = render_depth_and_opacity(sigma, delta, z)
print("Test — First sample opaque")
print(f"Computed C = {C3:.6f}, depth = {depth3:.6f}")


Test — First sample opaque
Computed C = 1.000000, depth = 3.000000


We notice that at Z = 3.0 sigma has value </br> 
The value of sigma affects the accuracy of Depth map, here we notice when sigma increases at this value predicts it well

## Experiment of depth could be zero but depends on C

In [5]:
sigma = np.zeros(10)
delta = np.ones(10)
z = np.arange(1, 11)
print(z)
C2, depth2 = render_depth_and_opacity(sigma, delta, z)
print("Test  — Zero density")
print(f"Computed C = {C2:.6f}, depth = {depth2:.6f}")


[ 1  2  3  4  5  6  7  8  9 10]
Test  — Zero density
Computed C = 0.000000, depth = 0.000000


In [6]:
sigma = np.zeros(10)
sigma[0] = 1000
print(sigma)
delta = np.ones(10)
z = np.arange(1, 11) * 0.001
print(z)
C2, depth2 = render_depth_and_opacity(sigma, delta, z)
print("Test  — Zero density")
print(f"Computed C = {C2:.6f}, depth = {depth2:.6f}")


[1000.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
[0.001 0.002 0.003 0.004 0.005 0.006 0.007 0.008 0.009 0.01 ]
Test  — Zero density
Computed C = 1.000000, depth = 0.001000


We notice that almost the tests their depth almost zero but <br>
<li>in the first, free space and we can see its C = 0
<li>in the Second, occupied space and we can see its C = 1

## We can infer that C could be see as certainity of this pixel occupied at this depth